# Побудова оцінок та довірчих інтервалів

Будуємо довірчі інтервали для `n = 100, 10_000, 1_000_000`
Рівень довіри = `0,99` (рівень значущості = `0.01`

#### Довірчі інтервали для м.с. та дисперсії у випадку нормально розподіленої генеральної сукупності.
##### Перша частина леми 1. (Пункт А):
Вибіркове математичне сподівання: $\hat{a}_{n}=\frac {1}{n}\sum^{n}_{k=1}X_k$
Вибіркова дисперсія: $\hat{\sigma}^2_{n}=\frac {1}{n}\sum^{n}_{k=1}(X_k-\hat{a}_{n})^2$
Дов інтервал $a \in a_n \pm \frac{z_{\gamma}  \sigma_n}{\sqrt{n}}$
К-ть ітерацій: $n$

In [48]:
import math

import numpy as np
from scipy.stats import chi2, t, norm

In [49]:
def calc_sample_variance(samples, sample_mean: float):
    squared_variance = 0.
    for sample in samples:
        squared_variance += (sample - sample_mean) ** 2

    squared_variance /= (len(samples) - 1)
    return math.sqrt(squared_variance)


def calc_sample_mean(samples):
    mean = 0.
    for sample in samples:
        mean += sample
    return mean / len(samples)

def calc_s_squared(samples, sample_mean):
    s_squared = 0
    for sample in samples:
        s_squared += (sample - sample_mean) ** 2

    s_squared /= (len(samples) - 1)
    return s_squared

# generating samples with help of N(0,1) distribution
def gen_samples(size: int):
    samples = size * [None]
    for i in range(size):
        samples[i] = np.random.normal(0, 1)
    return samples

In [50]:
samples_num = [100, 10_000, 1_000_000]
samples_list = len(samples_num) * [None]
alfa = 0.01
gama = 1 - alfa

# calculating 0.99 quantiles
print("   samples_num:{0}".format(samples_num))

   samples_num:[100, 10000, 1000000]


In [51]:
# calculating Confidence Interval for the EXPECTATION(sigma^2) under the assumption:
# - observed random value ~ N(0,1)
# - sample variance is unknown
def calculate_ci_1(samples):
    sample_mean = calc_sample_mean(samples)
    n_sq_root = math.sqrt(len(samples))
    s_squared = calc_s_squared(samples, sample_mean)

    g = t.ppf((1 + gama) / 2, df=len(samples)-1)
    return [sample_mean - math.sqrt(s_squared) * g/ n_sq_root, sample_mean + math.sqrt(s_squared) * g / n_sq_root]


# calculating Confidence Interval for the EXPECTATION under assumption:
# - observed value distribution is unknown
def calculate_ci_2(samples, gama):  # варіант в
    sample_mean = calc_sample_mean(samples)
    sample_variance = calc_sample_variance(samples, sample_mean)

    n_sq_root = math.sqrt(len(samples))

    c = norm.ppf((1 + gama) / 2)
    return [sample_mean - sample_variance * c/ n_sq_root, sample_mean + sample_variance * c / n_sq_root]


# calculating Confidence Interval for the variance under assumption:
# - observed random value ~ N(0,1)
def calculate_ci_3(samples, gama):
    sample_mean = calc_sample_mean(samples)
    g_1 = chi2.ppf((1 - gama) / 2, df=len(samples)-1)
    g_2 = chi2.ppf((1 + gama) / 2, df=len(samples)-1)

    squared_difference = 0
    for sample in samples:
        squared_difference = (sample - sample_mean) ** 2

    return [squared_difference / g_2, squared_difference / g_1]


def calculate_1_integral():
    pass


def calculate_2_integral():
    pass


def calculate_3_integral():
    pass


In [52]:
for i, sample_size in enumerate(samples_num):
    samples_list[i] = gen_samples(sample_size)
    sample_mean = calc_sample_mean(samples_list[i])
    sample_variance = calc_sample_variance(samples_list[i], sample_mean)
    # print("samples: {0}".format(samples_list[i]))
    print(f"Sample_mean_[{i+1}]: {sample_mean}, Sample_variance_{i+1}: {sample_variance}")
    print(f"CI of expected value (1): {calculate_ci_1(samples_list[i])}")
    print(f"CI of expected value (2): {calculate_ci_2(samples_list[i], gama)}")
    print(f"CI of variance (3)      : {calculate_ci_3(samples_list[i], gama)}")
    # print("CI of expected value (2): {0}".format(calculate_ci_2(samples_list[i])))

    calculate_1_integral()
    calculate_2_integral()
    calculate_3_integral()



Sample_mean_[1]: 0.047777566999129634, Sample_variance_1: 0.9123888582375852
CI of expected value (1): [-0.19185274056289472, 0.287407874561154]
CI of expected value (2): [-0.18723822872885992, 0.2827933627271192]
CI of variance (3)      : [0.005083687521965348, 0.010623428929215428]
Sample_mean_[2]: 0.011888655900747997, Sample_variance_2: 1.007203233836424
CI of expected value (1): [-0.014060133490729652, 0.037837445292225644]
CI of expected value (2): [-0.014055180142702765, 0.03783249194419876]
CI of variance (3)      : [5.425076673675621e-05, 5.835125890838432e-05]
Sample_mean_[3]: 0.0003784940177670839, Sample_variance_3: 0.9987072997704358
CI of expected value (1): [-0.0021940104208555107, 0.0029509984563896785]
CI of expected value (2): [-0.002194005510649801, 0.0029509935461839686]
CI of variance (3)      : [1.0669111176552241e-06, 1.0747125403618728e-06]
